#### To Do
- download each csv file to ../tmp folder
- add ../tmp to strblob

In [11]:
import pyspark
from pyspark.sql import SparkSession
from prefect import flow, task
from google.cloud import storage

In [12]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [13]:
catch_dtype = {'common': str, 'strat_id': str, 'psu_id': int, 
                'id_code': int, 'MODE_FX': int, 'AREA_X': int, 
                'ST': int, 'SUB_REG': int, 'WAVE': int, 
                'year': int, 'month': int, 'kod': str, 
                'SP_CODE': float, 'CLAIM': float, 'RELEASE': float, 
                'HARVEST': float, 'CLAIM_UNADJ': int, 'HARVEST_UNADJ': int, 
                'RELEASE_UNADJ': int, 'tot_len_a': float, 'wgt_a': float, 
                'tot_len_b1': float, 'wgt_b1': float, 'tot_cat': float, 
                'wgt_ab1': float, 'tot_len': float, 'Landing': float, 
                'VAR_ID': str, 'ARX_METHOD': float, 'ALT_FLAG': int, 
                'fl_reg': int, 'wp_catch_precal': float, 'wp_int': float, 
                'wp_catch': float, 'date_published': str}

trip_dtype = {'prim2_common': str, 'prim1_common': str, 'strat_id': str, 
              'psu_id': int, 'ADD_HRS': float, 'AREA': int, 
              'AREA_X': int, 'BOAT_HRS': float, 'CATCH': int, 
              'CNTRBTRS': int, 'CNTY': int, 'CNTY_RES': int, 
              'COASTAL': str, 'FFDAYS2': int, 'FFDAYS12': int, 
              'FIRST': float, 'HRSF': float, 'ID_CODE': int, 
              'INTSITE': int, 'MODE_F': int, 'MODE_FX': int, 
              'NUM_TYP2': int, 'NUM_TYP3': int, 'NUM_TYP4': int, 
              'NUM_TYP6': float, 'ON_LIST': float, 'PARTY': float, 
              'REG_RES': float, 'ST': int, 'ST_RES': int, 
              'SUB_REG': int, 'TELEFON': int, 'WAVE': int, 
              'YEAR': int, 'ASG_CODE': int, 'month': int, 
              'kod': str, 'MODE_ASG': float, 'new_list': float, 
              'PRT_CODE': int, 'CELLTYPE': float, 'fshinsp_a': float, 
              'num_fish_a': float, 'fl_reg': int, 'ADD_PH': int, 
              'AREA_NC': float, 'COUNTY': str, 'DATE1': str, 
              'DIST': int, 'F_BY_P': int, 'GEAR': int, 
              'MODE2001': float, 'MUNI_RES': float, 'MUNI_TRP': float, 
              'PRIM1': float, 'PRIM2': float, 'PVT_RES': int, 
              'RIG': int, 'SEP_FISH': int, 'TIME': int, 
              'ZIP': float, 'AGE': float, 'wp_int': float, 
              'VAR_ID': str, 'ARX_METHOD': float, 'ALT_FLAG': int, 
              'LEADER': int, 'date_published': str}

size_dtype = {'AREA_X': int, 'ID_CODE': int, 'MODE_FX': int, 
              'ST': int, 'SUB_REG': int, 'WAVE': int, 
              'YEAR': int, 'month': int, 'kod': str, 
              'SP_CODE': float, 'LNGTH': float, 'WGT': float, 
              'lngth_imp': int, 'wgt_imp': int, 'strat_id': str, 
              'psu_id': int, 'common': str, 'wgt_unadj': float, 
              'wp_size': float, 'l_in_bin': float, 'l_cm_bin': float, 
              'VAR_ID': str, 'ARX_METHOD': float, 'ALT_FLAG': int, 
              'date_published': str}

In [14]:
@task()
def create_df(folder: str, bucket) -> list:
    """Get list of zip folders in GCS bucket"""
    lsblob = list(bucket.list_blobs(prefix=f"{folder}"))  # get list of blobs in zip folder of bucket
    lsblob = [l.name for l in lsblob]  # only return the filename from the blobs
    strblob = ','.join(lsblob)
    print(strblob)

    df = spark.read.csv(strblob)
    return df

c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\tasks.py:275: UserWarning: A task named 'create_df' and defined at 'C:\Users\jlopez\AppData\Local\Temp\ipykernel_23604\1023419110.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


In [15]:
@flow(log_prints=False)
def write_bq(folder: str = "catch", bucket_name: str = "de_project_bucket") -> None:
    """Process blob in gcs bucket"""
    storage_client = storage.Client.from_service_account_json("../creds.json")
    # bucket_name = "de_project_bucket"  # parameterize this
    bucket = storage_client.get_bucket(bucket_name)

    df = create_df(folder, bucket)
    print(df.head())

c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\flows.py:214: UserWarning: A flow named 'write-bq' and defined at 'C:\Users\jlopez\AppData\Local\Temp\ipykernel_23604\1671885404.py:1' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


In [16]:
if __name__ == '__main__':
    folder = "size"
    bucket_name = "de_project_bucket"
    write_bq(folder, bucket_name)  



22:59:50.417 | INFO    | prefect.engine - Created flow run 'hot-mule' for flow 'write-bq'

22:59:51.182 | INFO    | Flow run 'hot-mule' - Created task run 'create_df-0' for task 'create_df'

22:59:51.186 | INFO    | Flow run 'hot-mule' - Executing 'create_df-0' immediately...

size/size_19811.csv,size/size_19812.csv,size/size_19813.csv,size/size_19814.csv,size/size_19815.csv,size/size_19816.csv,size/size_19821.csv,size/size_19822.csv,size/size_19823.csv,size/size_19824.csv,size/size_19825.csv,size/size_19826.csv,size/size_19831.csv,size/size_19832.csv,size/size_19833.csv,size/size_19834.csv,size/size_19835.csv,size/size_19836.csv,size/size_19841.csv,size/size_19842.csv,size/size_19843.csv,size/size_19844.csv,size/size_19845.csv,size/size_19846.csv,size/size_19851.csv,size/size_19852.csv,size/size_19853.csv,size/size_19854.csv,size/size_19855.csv,size/size_19856.csv,size/size_19861.csv,size/size_19862.csv,size/size_19863.csv,size/size_19864.csv,size/size_19865.csv,size/size_19866.csv,size/size_19871.csv,size/size_19872.csv,size/size_19873.csv,size/size_19874.csv,size/size_19875.csv,size/size_19876.csv,size/size_19881.csv,size/size_19882.csv,size/size_19883.csv,size/size_19884.csv,size/size_19885.csv,size/size_19886.csv,size/size_19891.csv,size/size_19892.csv,

22:59:53.231 | ERROR   | Task run 'create_df-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\engine.py", line 1545, in orchestrate_task_run
    result = await call.aresult()
             ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 181, in aresult
    return await asyncio.wrap_future(self.future)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 194, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jlopez\AppData\Local\Temp\ipykernel_23604\1023419110.py", line 9, in create_df
    df = spark.read.csv(strblob)
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\pyspark\sql\readwriter.py", line 535, in csv
    return self._df(self._jreader.csv(self._spark._sc._jvm.PythonUtils.toSeq(path)))
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\py4j\java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\pyspark\sql\utils.py", line 196, in deco
    raise converted from None
pyspark.sql.utils.AnalysisException: Path does not exist: file:/c:/Users/jlopez/Dropbox/de-project/notebooks/size/size_19811.csv,size/size_19812.csv,size/size_19813.csv,size/size_19814.csv,size/size_19815.csv,size/size_19816.csv,size/size_19821.csv,size/size_19822.csv,size/size_19823.csv,size/size_19824.csv,size/size_19825.csv,size/size_19826.csv,size/size_19831.csv,size/size_19832.csv,size/size_19833.csv,size/size_19834.csv,size/size_19835.csv,size/size_19836.csv,size/size_19841.csv,size/size_19842.csv,size/size_19843.csv,size/size_19844.csv,size/size_19845.csv,size/size_19846.csv,size/size_19851.csv,size/size_19852.csv,size/size_19853.csv,size/size_19854.csv,size/size_19855.csv,size/size_19856.csv,size/size_19861.csv,size/size_19862.csv,size/size_19863.csv,size/size_19864.csv,size/size_19865.csv,size/size_19866.csv,size/size_19871.csv,size/size_19872.csv,size/size_19873.csv,size/size_19874.csv,size/size_19875.csv,size/size_19876.csv,size/size_19881.csv,size/size_19882.csv,size/size_19883.csv,size/size_19884.csv,size/size_19885.csv,size/size_19886.csv,size/size_19891.csv,size/size_19892.csv,size/size_19893.csv,size/size_19894.csv,size/size_19895.csv,size/size_19896.csv

22:59:53.359 | ERROR   | Task run 'create_df-0' - Finished in state Failed('Task run encountered an exception: pyspark.sql.utils.AnalysisException: Path does not exist: file:/c:/Users/jlopez/Dropbox/de-project/notebooks/size/size_19811.csv,size/size_19812.csv,size/size_19813.csv,size/size_19814.csv,size/size_19815.csv,size/size_19816.csv,size/size_19821.csv,size/size_19822.csv,size/size_19823.csv,size/size_19824.csv,size/size_19825.csv,size/size_19826.csv,size/size_19831.csv,size/size_19832.csv,size/size_19833.csv,size/size_19834.csv,size/size_19835.csv,size/size_19836.csv,size/size_19841.csv,size/size_19842.csv,size/size_19843.csv,size/size_19844.csv,size/size_19845.csv,size/size_19846.csv,size/size_19851.csv,size/size_19852.csv,size/size_19853.csv,size/size_19854.csv,size/size_19855.csv,size/size_19856.csv,size/size_19861.csv,size/size_19862.csv,size/size_19863.csv,size/size_19864.csv,size/size_19865.csv,size/size_19866.csv,size/size_19871.csv,size/size_19872.csv,size/size_19873.csv,size/size_19874.csv,size/size_19875.csv,size/size_19876.csv,size/size_19881.csv,size/size_19882.csv,size/size_19883.csv,size/size_19884.csv,size/size_19885.csv,size/size_19886.csv,size/size_19891.csv,size/size_19892.csv,size/size_19893.csv,size/size_19894.csv,size/size_19895.csv,size/size_19896.csv\n')

22:59:53.365 | ERROR   | Flow run 'hot-mule' - Encountered exception during execution:
Traceback (most recent call last):
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\engine.py", line 673, in orchestrate_flow_run
    result = await flow_call.aresult()
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 181, in aresult
    return await asyncio.wrap_future(self.future)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 194, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jlopez\AppData\Local\Temp\ipykernel_23604\1671885404.py", line 8, in write_bq
    df = create_df(folder, bucket)
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\tasks.py", line 474, in __call__
    return enter_task_run_engine(
           ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\engine.py", line 976, in enter_task_run_engine
    return from_sync.wait_for_call_in_loop_thread(begin_run)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\_internal\concurrency\api.py", line 137, in wait_for_call_in_loop_thread
    return call.result()
           ^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 173, in result
    return self.future.result(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\concurrent\futures\_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\concurrent\futures\_base.py", line 401, in __get_result
    raise self._exception
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 218, in _run_async
    result = await coro
             ^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\engine.py", line 1131, in get_task_call_return_value
    return await future._result()
           ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\futures.py", line 240, in _result
    return await final_state.result(raise_on_failure=raise_on_failure, fetch=True)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\states.py", line 91, in _get_state_result
    raise await get_state_exception(state)
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\engine.py", line 1545, in orchestrate_task_run
    result = await call.aresult()
             ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 181, in aresult
    return await asyncio.wrap_future(self.future)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jlopez\AppData\Local\Continuum\anaconda3\envs\de_project\Lib\site-packages\prefect\_internal\concurrency\calls.py", line 194, in _run_sync
    result = self.fn(*self.args, **self.kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jlopez\AppData\Local\Temp\ipykernel_23604\10

22:59:53.481 | ERROR   | Flow run 'hot-mule' - Finished in state Failed('Flow run encountered an exception. pyspark.sql.utils.AnalysisException: Path does not exist: file:/c:/Users/jlopez/Dropbox/de-project/notebooks/size/size_19811.csv,size/size_19812.csv,size/size_19813.csv,size/size_19814.csv,size/size_19815.csv,size/size_19816.csv,size/size_19821.csv,size/size_19822.csv,size/size_19823.csv,size/size_19824.csv,size/size_19825.csv,size/size_19826.csv,size/size_19831.csv,size/size_19832.csv,size/size_19833.csv,size/size_19834.csv,size/size_19835.csv,size/size_19836.csv,size/size_19841.csv,size/size_19842.csv,size/size_19843.csv,size/size_19844.csv,size/size_19845.csv,size/size_19846.csv,size/size_19851.csv,size/size_19852.csv,size/size_19853.csv,size/size_19854.csv,size/size_19855.csv,size/size_19856.csv,size/size_19861.csv,size/size_19862.csv,size/size_19863.csv,size/size_19864.csv,size/size_19865.csv,size/size_19866.csv,size/size_19871.csv,size/size_19872.csv,size/size_19873.csv,size/size_19874.csv,size/size_19875.csv,size/size_19876.csv,size/size_19881.csv,size/size_19882.csv,size/size_19883.csv,size/size_19884.csv,size/size_19885.csv,size/size_19886.csv,size/size_19891.csv,size/size_19892.csv,size/size_19893.csv,size/size_19894.csv,size/size_19895.csv,size/size_19896.csv\n')

AnalysisException: Path does not exist: file:/c:/Users/jlopez/Dropbox/de-project/notebooks/size/size_19811.csv,size/size_19812.csv,size/size_19813.csv,size/size_19814.csv,size/size_19815.csv,size/size_19816.csv,size/size_19821.csv,size/size_19822.csv,size/size_19823.csv,size/size_19824.csv,size/size_19825.csv,size/size_19826.csv,size/size_19831.csv,size/size_19832.csv,size/size_19833.csv,size/size_19834.csv,size/size_19835.csv,size/size_19836.csv,size/size_19841.csv,size/size_19842.csv,size/size_19843.csv,size/size_19844.csv,size/size_19845.csv,size/size_19846.csv,size/size_19851.csv,size/size_19852.csv,size/size_19853.csv,size/size_19854.csv,size/size_19855.csv,size/size_19856.csv,size/size_19861.csv,size/size_19862.csv,size/size_19863.csv,size/size_19864.csv,size/size_19865.csv,size/size_19866.csv,size/size_19871.csv,size/size_19872.csv,size/size_19873.csv,size/size_19874.csv,size/size_19875.csv,size/size_19876.csv,size/size_19881.csv,size/size_19882.csv,size/size_19883.csv,size/size